In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pyspark.sql.types as st
from datetime import date

spark = SparkSession.builder.getOrCreate()

daily_country_confirmed_cases = "./data/Daily_Country_Wise_Confirmed_Cases.csv"

daily_df = spark.read.csv(daily_country_confirmed_cases,inferSchema=True,header=True)


In [32]:
# Change to importing this function
def date_check():
    today = date.today()
    return str(today)

from datetime import datetime, timedelta

def date_minus_weeks(x:int) -> str:
    """
    Returns a timedelta STRING
    """
    return str(date.today() - timedelta(weeks = x))

In [52]:
# Showing the change of cases over a month
daily_df.select('Country',daily_df[-1]).where(daily_df[-1] >= 1).show()

+-------------+----------+
|      Country|2022-07-01|
+-------------+----------+
|United States|        64|
|      Germany|        85|
|        Italy|        33|
|  Switzerland|         8|
|      Austria|        17|
+-------------+----------+



In [44]:
# Shows the change over 1 week
daily_df.select('Country', daily_df[date_minus_weeks(2)],daily_df[date_minus_weeks(1)]).where(daily_df[date_minus_weeks(1)] > 10).show()

+-------------+----------+----------+
|      Country|2022-06-22|2022-06-29|
+-------------+----------+----------+
|     Portugal|        13|        18|
|United States|        14|        45|
|      Germany|        54|        35|
|       Canada|         0|        12|
|       Brazil|         2|        16|
+-------------+----------+----------+



In [43]:
daily_df.select(
    'Country',
    daily_df[date_minus_weeks(7)],
    daily_df[date_minus_weeks(6)],
    daily_df[date_minus_weeks(5)],
    daily_df[date_minus_weeks(4)],
    daily_df[date_minus_weeks(3)],
    daily_df[date_minus_weeks(2)],
    daily_df[date_minus_weeks(1)])\
        .show()

+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             Country|2022-05-18|2022-05-25|2022-06-01|2022-06-08|2022-06-15|2022-06-22|2022-06-29|
+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             England|         2|         7|         5|         0|         0|         0|         0|
|            Portugal|        11|        10|        19|        25|        10|        13|        18|
|               Spain|         7|         8|         0|        34|         0|         0|         0|
|       United States|         1|         2|         0|         5|        12|        14|        45|
|             Germany|         0|         1|         0|        13|        45|        54|        35|
|             Belgium|         0|         2|         5|         8|        27|         0|         0|
|              Sweden|         0|         1|         0|         0|         4|         0|         0|


In [77]:
# Worldwide total, sum of all.
from functools import reduce

def column_add(a,b):
     return  a.__add__(b)
agg1_df = daily_df.drop('Country')

# def column_add(a:int,b:int):
#      if type(a) == int and type(b) == int:
#           return  a+b
#      else:
#           pass
agg1_df = agg1_df.withColumn('total', reduce(column_add, (agg1_df[col] for col in agg1_df.columns)))
agg1_df.groupBy().sum().show()

# daily_df.withColumn('totals', reduce(column_add, (daily_df[col] for col in daily_df.columns))).show()

# daily_df.drop(daily_df['Country']).withColumn('total_col', reduce(column_add, (daily_df[col] for col in daily_df.columns))).show()



+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+
|sum(2022-05-06)|sum(2022-05-12)|sum(2022-05-13)|sum(2022-05-15)|sum(2022-05-17)|sum(2022-05-18)|sum(2022-05-19)|sum(2022-05-20)|sum(2022-05-21)|sum(2022-05-23)|sum(2022-05-24)|sum(2022-05-25)|sum(2022-05-26)|sum(2022-0

In [ ]:
# Create dataframe for region totals weekly totals and sums

africa = ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde", "Central African Republic",
    "Chad", "Comoros", "Cote d'Ivoire", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea",
    "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi",
    "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo", "Reunion",
    "Rwanda", "Saint Helena", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", 
    "Sudan", "Swaziland", "Tanzania", "Togo", "Tunisia", "Uganda", "Western Sahara", "Zambia", "Zimbabwe"]

asia = ["Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Burma", "Cambodia", "China", "Cyprus",
    "East Timor", "Georgia", "Hong Kong", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Kuwait",
    "Kyrgyzstan", "Laos", "Lebanon", "Macau", "Malaysia", "Maldives", "Mongolia", "Nepal", "North Korea", "Oman", "Pakistan", "Philippines",
    "Qatar", "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Taiwan", "Tajikistan", "Thailand", "Turkey", "Turkmenistan",
    "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen"]

caribbean = ["Anguilla", "Antigua and Barbuda", "Aruba", "The Bahamas", "Barbados", "Bermuda", "British Virgin Islands", "Cayman Islands",
    "Cuba", "Dominica", "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "Netherlands Antilles",
    "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Trinidad and Tobago", "Turks and Caicos Islands", "U.S. Virgin Islands"]

central_america = ["Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Nicaragua", "Panama"]

europe = ["Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", 
    "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Gibraltar", "Greece", "Holy See", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia",
    "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Russia", 
    "San Marino", "Slovak Republic", "Slovenia", "Spain", "Serbia", "Serbia and Montenegro", "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Northern Ireland", "Wales"]

north_america = ["Canada", "Greenland", "Mexico", "Saint Pierre and Miquelon", "United States"]

oceania = ["American Samoa", "Australia", "Christmas Island", "Cocos (Keeling) Islands", "Cook Islands", "Federated States of Micronesia", "Fiji", "French Polynesia", 
    "Guam", "Kiribati", "Marshall Islands", "Nauru", "New Caledonia", "New Zealand", "Niue", "Northern Mariana Islands", "Palau", "Papua New Guinea", "Pitcairn Islands",
    "Samoa", "Solomon Islands", "Tokelau", "Tonga", "Tuvalu", "Vanuatu", "Wallis and Futuna Islands"]

south_america = ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Falkland Islands", "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay",
    "Venezuela"]

In [ ]:
# Total daily average.